In [47]:
import pandas as pd
import folium

In [48]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

### Task 1: Mark all launch sites on a map

In [49]:
spacex_df = pd.read_csv(r"F:\Data_Science_Jupyter_Files\spacex_launch_geo.csv")

In [50]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, we can take a look at what are the coordinates for each site.

In [51]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]
launch_sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()
launch_sites_df = launch_sites_df[["Launch Site", "Lat", "Long"]]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.
We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [52]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate.

In [53]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color="red", fill=True).add_child(folium.Popup("NASA Johnson Space Center"))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon = DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:black;"><b>%s</b></div>' % 'NASA JSC',
    )
    )
site_map.add_child(circle)
site_map.add_child(marker)

TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

In [54]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
circle = folium.Circle(nasa_coordinate, radius=10000, color="black", fill=True).add_child(folium.Popup("NASA Johnson Space Center"))
launch_site_1_circle = folium.Circle([28.562302, -80.577356], radius=10000, color="blue", fill=True).add_child(folium.Popup("CCAFS LC-40"))
launch_site_2_circle = folium.Circle([28.563197, -80.576820], radius=10000, color="blue", fill=True).add_child(folium.Popup("CCAFS SLC-40"))
launch_site_3_circle = folium.Circle([28.573255, -80.646895], radius=10000, color="blue", fill=True).add_child(folium.Popup("KSC LC-39A"))
launch_site_4_circle = folium.Circle([34.632834, -120.610745], radius=10000, color="blue", fill=True).add_child(folium.Popup("VAFB SLC-4E"))

site_map.add_child(circle)
site_map.add_child(launch_site_1_circle)
site_map.add_child(launch_site_2_circle)
site_map.add_child(launch_site_3_circle)
site_map.add_child(launch_site_4_circle)

### Task 2: Mark the success/failed launches for each site on the map

Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [55]:
# Let's first create a MarkerCluster object
marker_cluster = MarkerCluster()

#### TODO: Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

In [56]:
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

#### TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

In [57]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)
# Loop through each launch record in the DataFrame
for index, record in spacex_df.iterrows():
    # Get the coordinates
    location = [record["Lat"], record["Long"]]
    # Create a Marker with custom icon color
    marker = folium.Marker(
        location = location,
        icon = folium.Icon(
            color = "white",  # background of the marker (default)
            icon_color = record["marker_color"],  # the color of the icon text (green/red)
            icon = "rocket",  # rocket symbol from Font Awesome
            prefix = "fa"  # tells folium to use Font Awesome icons
        )
    )

    # Add marker to the cluster
    marker_cluster.add_child(marker)


# Add marker cluster to the map
site_map.add_child(marker_cluster)

# Show the map
site_map

### TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [58]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator="Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat: ",
    lat_formatter=formatter,
    long_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [59]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    # Converts latitude and longitude from degrees to radians (because trigonometric functions in Python use radians).
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Finds the difference between the two locations in both latitude and longitude.
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # This is part of the Haversine formula, which calculates the "straight-line" distance over the Earth’s curved surface.
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    # Haversine formula math — it calculates an angle that helps figure out the distance.
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

TODO: Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [60]:
launch_site = (28.563197, -80.576820)
coastline_point = (28.56332, -80.56799)

In [61]:
distance = calculate_distance(*launch_site, *coastline_point)

In [62]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property

distance_marker = folium.Marker(
    coastline_point,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
).add_to(site_map)
site_map

In [63]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
folium.PolyLine(
    locations = [launch_site, coastline_point], # List of coordinates
    color = "purple", # Line color
    weight = 2,
    opacity = 1
).add_to(site_map)
site_map